In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait # 대기툴
from selenium.webdriver.support import expected_conditions as EC # 대기툴
import requests
from bs4 import BeautifulSoup
import time
import os
import random
import csv

In [2]:
# melon 열기
url="https://www.melon.com/"
headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36"}
res=requests.get(url,headers=headers)
browser=webdriver.Chrome()# 새창열기
browser.maximize_window()# 창화면 확대
browser.get(url)
time.sleep(1)

In [3]:
# 차트열기
elem=browser.find_element(By.XPATH,'//*[@id="gnb_menu"]/ul[1]/li[1]/a')
elem.click()

In [4]:
# 데이터 받기(순위,커버아트,제목,아티스트,좋아요)-1위
soup=BeautifulSoup(browser.page_source,'lxml')# html소스 bs로 변경

trs=soup.find_all('tr',{'class':'lst50'})

print(trs[0].find('span',{'class':'rank'}).text.strip())# 1위 순위
print(trs[0].find('img')['src'])# 1위 커버아트
print(trs[0].find('div',{'class':'ellipsis rank01'}).text.strip())# 1위 제목
print(trs[0].find('span',{'class':'checkEllipsis'}).text.strip())# 1위 아티스트
print(int(trs[0].find('span',{'class':'cnt'}).text.split('\n')[-1].strip().replace(',','')))# 1위 좋아요수


1
https://cdnimg.melon.co.kr/cm2/album/images/121/60/000/12160000_20251014164646_500.jpg/melon/resize/120/quality/80/optimize
Good Goodbye
화사 (HWASA)
72897


In [6]:
# 데이터 받기(순위,커버아트,제목,아티스트,좋아요)-전체
for tr in trs:
    rank=tr.find('span',{'class':'rank'}).text.strip()
    albemArt=tr.find('img')['src']
    title=tr.find('div',{'class':'ellipsis rank01'}).text.strip()
    artist=tr.find('span',{'class':'checkEllipsis'}).text.strip()
    likes=int(tr.find('span',{'class':'cnt'}).text.split('\n')[-1].strip().replace(',',''))
    print(f'{rank}\t{albemArt}\n{title}\t{artist}\t{likes}')
    print('-'*50)

1	https://cdnimg.melon.co.kr/cm2/album/images/121/60/000/12160000_20251014164646_500.jpg/melon/resize/120/quality/80/optimize
Good Goodbye	화사 (HWASA)	72897
--------------------------------------------------
2	https://cdnimg.melon.co.kr/cm2/album/images/121/63/032/12163032_20251015164500_500.jpg/melon/resize/120/quality/80/optimize
타임캡슐	다비치	44001
--------------------------------------------------
3	https://cdnimg.melon.co.kr/cm2/album/images/121/49/391/12149391_20251013150421_500.jpg/melon/resize/120/quality/80/optimize
Blue Valentine	NMIXX	94145
--------------------------------------------------
4	https://cdnimg.melon.co.kr/cm2/album/images/123/92/184/12392184_20251117102938_500.jpg/melon/resize/120/quality/80/optimize
ONE MORE TIME	ALLDAY PROJECT	40161
--------------------------------------------------
5	https://cdnimg.melon.co.kr/cm/album/images/022/19/671/2219671_500.jpg/melon/resize/120/quality/80/optimize
첫 눈	EXO	311660
--------------------------------------------------
6	https://

In [ ]:
# 좋아요 10만이상 출력
for tr in trs:
    rank=tr.find('span',{'class':'rank'}).text.strip()
    albemArt=tr.find('img')['src']
    title=tr.find('div',{'class':'ellipsis rank01'}).text.strip()
    artist=tr.find('span',{'class':'checkEllipsis'}).text.strip()
    likes=int(tr.find('span',{'class':'cnt'}).text.split('\n')[-1].strip().replace(',',''))
    if likes>=100000:
        print(f'{rank}\t{albemArt}\n{title}\t{artist}\t{likes}')
        print('-'*50)

In [8]:
# csv 저장(순위,커버아트,제목,아티스트,좋아요)-전체 + 이미지 저장
f= open('melonTop100.csv',"w",encoding="utf-8-sig",newline="")
writer=csv.writer(f)
writer.writerow(['순위','커버아트','제목','아티스트','좋아요수'])
for i,tr in enumerate(trs):
    rank=tr.find('span',{'class':'rank'}).text.strip()
    albemArt=tr.find('img')['src']
    title=tr.find('div',{'class':'ellipsis rank01'}).text.strip()
    artist=tr.find('span',{'class':'checkEllipsis'}).text.strip()
    likes=int(tr.find('span',{'class':'cnt'}).text.split('\n')[-1].strip().replace(',',''))
    writer.writerow([rank,albemArt,title,artist,likes])
    
    # 앨범아트 따로 폴더에 파일로 저장
    os.makedirs('melonTop100',exist_ok=True)# 폴더가 없으면 생성함
    albemArt_url=requests.get(tr.find('img')['src'],headers=headers)
    with open(f'melonTop100/melonRank_{i+1}.jpg','wb') as ff:# f,ff따로
        ff.write(albemArt_url.content)# 파일로 저장
f.close()
# 폴더를 만들어 파일저장
    